# Анализ сайта gismeteo.ru

Для выполнения итогового задания в качестве источника данных метеосводок был выбран сервис Дневник сайта Gismeteo.ru, в котором можно бесплатно получить исторические данные о погоде, начиная с 1997 года. Необходимо получить данные с 2008 по 2017.

![Внешний вид сервиса Дневник от gismeteo](images/gismeteo_example.png)

В реальных условиях для получения исторических метео-данных и прогнозов на долгий срок необходимо использовать ресурсы, такие как:
- [Gismeteo](https://www.gismeteo.ru/api/) - российский сервис; в документации API нет исторических данных, а также указана возможность получения прогноза погоды до 10 дней, однако на самом сайте есть возможно получить [исторические данные](https://www.gismeteo.ru/diary/4368/2008/12/1) и прогноз более, чем на [месяц вперёд](https://www.gismeteo.ru/weather-moscow-4368/month/).
- [Яндекс.Погода](https://yandex.ru/dev/weather/) - российский сервис; прогноз на 10 дней и исторические данные за 10 лет.
- [AccuWeather](https://developer.accuweather.com/packages) - прогноз до 15 дней.
- [Meteostat](https://dev.meteostat.net/api/point/daily.html) - исторические данные за 10 лет.
- [Open-Meteo](https://open-meteo.com/en) - прогноз до 7 дней и исторические данные за 60 лет.
- [OpenWeather](https://openweathermap.org/price#history) - прогноз до 16 дней и исторические данные за 40 лет.
- [Weather API](https://www.weatherapi.com/pricing.aspx)- прогноз до 14 дней и исторические данные с 2010 года.
- [Weatherbit.io](https://www.weatherbit.io/pricing) - прогноз до 16 дней и исторические данные за 20 лет.
- [meteoblue](https://content.meteoblue.com/en/business-solutions/weather-apis/packages-api) - прогноз до 14 дней и исторические данные за 35 лет.
- [weatherstack](https://weatherstack.com/product) - прогноз до 14 дней и исторические данные с 2008 года.

После исполнения скрипта на загрузку данных из сайта автоматизированной информационной системы государственного мониторинга водных объектов Российской Федерации (далее - "АИС ГМВО") **src\preparation\scrape_water_level.py**, в **data\raw\water_posts_data.json** будет находиться словарь **{uid: наименование поста наблюдения}**. Наименования представлены в виде: *название водного объекта* - *название населённого пункта*

## Получение списка населённых пунктов с постами наблюдения

### Проверка и установка рабочей директории, должен быть корень проекта

In [1]:
%pwd

'C:\\Users\\Kuroha\\source\\repos_py\\bauman_final_project\\notebooks'

In [2]:
%cd ..

C:\Users\Kuroha\source\repos_py\bauman_final_project


### Загрузка списка постов

In [3]:
from src.utils import *

if not is_data_exists(DATA_POSTS_RAW, is_raw=True):
    print('Данные с постами наблюдения АИС ГМВО не были загружены. Пожалуйста, '
          'выполните скрипт src\preparation\scrape_water_level.py'
          'и попробуйте ещё раз.')
else:
    file_posts = open_file(DATA_POSTS_RAW, is_raw=True)
    posts = json.loads(file_posts)
print(len(posts))
posts  # основная переменная, в которой будет храниться вся информация о гидрологических постах

28


{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска'},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска'},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска'},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска'},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска'},
 '09392': {'name': 'р.Чуня - пос.Стрелка Чуня',
  '

### Создание набора с очищенными названиями населённых пунктов

In [4]:
#test_name = next(iter(posts.values()))  # р.Подкаменная Тунгуска - пос.Чемдальск
#test_name = posts['09388']  # р.Подкаменная Тунгуска - факт.Усть-Камо
test_name = posts['09415']['name']  # р.Нижняя Тунгуска - факт.Большой Порог
test_name

'р.Нижняя Тунгуска - факт.Большой Порог'

In [5]:
import re

'''
Между видом и названием населённого пункта есть или точка, или пробел, используем [ \.]{1}
Учтено наличие в названии пункта пробела и тире
В БД gismeteo названия некоторых пунктов отличаются от таковых из сайта АИС ГМВО
'''
def clear_name(input_str):
    result = input_str.replace('Вельмо 2-е', 'Вельмо')
    result = result.replace('Вельмо2-е', 'Вельмо')
    result = result.replace('Стрелка Чуня', 'Стрелка-Чуня')
    result = result.replace('Большой порог', 'Большой Порог')
    result = re.search(r" - \w+[ \.]{1}([\w -]+)", result).group(1)
    return result

clear_name(test_name)

'Большой Порог'

In [6]:
for uid, item in posts.items():
    orig_name = item['name']
    posts[uid].update({'clean_name': clear_name(orig_name)})

posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Чемдальск'},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Ванавара'},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Усть-Камо'},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Байкит'},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'c

### Создание списка с уникальными населёнными пунктами

In [7]:
names = set([x['clean_name'] for x in posts.values()])
names

{'Байкит',
 'Большой Порог',
 'Бурный',
 'Ванавара',
 'Вельмо',
 'Верхнекарелина',
 'Ербогачен',
 'Ика',
 'Кербо',
 'Кислокан',
 'Кузьмовка',
 'Муторай',
 'Непа',
 'Подволошино',
 'Преображенка',
 'Светлана',
 'Стрелка-Чуня',
 'Суломай',
 'Тембенчи',
 'Тея',
 'Токма',
 'Тура',
 'Усть-Камо',
 'Учами',
 'Чемдальск'}

In [8]:
print(f'Число постов: {len(posts)}\nЧисло уникальных пунктов: {len(names)}')

Число постов: 28
Число уникальных пунктов: 25


## Проверка наличия исторических метео-данных

С получением метео-данных есть 2 проблемы:
1. Не все населённые пункты представлены в списке городов.
2. В некоторых пунктах за период с 2008 по 2017 не велись наблюдения метео-данных. При попытке их получить сайт показывает сообщение:
> Наблюдения метео-данных в данный период не велись.

![Сообщение об ошибке](images/gismeteo_error.png)

Необходимо обнаружить проблемные населённые пункты.

### Получение списка населённых пунктов с gismeteo

После открытия https://www.gismeteo.ru/diary/ и выбора страны Россия и области Красноярский край происходит 3 GET запроса:
![GET запросы](images/gismeteo_requests.png)

1. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/countries/?fr=sel  
Получение списка стран c ID и названиями.

2. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/districts/?country=156&fr=sel  
Получения списка областей c ID и названиями. В параметре country указан ID выбранной страны (156 - Россия).

3. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/cities/?district=301&fr=sel  
Получения списка городов c ID и названиями. В параметре district указан ID выбранной области (301 - Красноярский край, 303 - Иркутская область).

Во всех запросах **63466572668a39754a9ddf4c8b3437b0** - это стандартный api-токен, который не зависит от IP-адреса пользователя. Запросы возвращают данные в формате xml.
Учтём, что Подкаменная Тунгуска и Нижняя Тунгуска протекают в Красноярском крае и Иркутской области России, следовательно нужно запрашивать данные о пунктах в обоих субъектах.

In [9]:
from src.utils import get_url
import xmltodict

def get_cities(district):
    file_name = f'gismeteo_cities-{district}.xml'
    if is_data_exists(file_name, is_raw=True):
        cities = open_file(file_name, is_raw=True)
    else:
        url = f'https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/cities/?district={district}&fr=sel'
        r = get_url(url)
        cities = r.text
        write_data(file_name, data=cities, is_raw=True)
    cities = xmltodict.parse(cities)  # конвертация xml в dict
    # основные данные находятся в cities['document']['item']
    cities = cities['document']['item']
    return cities

cities = {}
target_districts = [301, 303]  # нужные области
cities_file_name = f'gismeteo_cities-{",".join([str(x) for x in target_districts])}.json'
if is_data_exists(cities_file_name, is_raw=True):
    cities = open_file(cities_file_name, is_raw=True)
    cities = json.loads(cities)
else:
    for district in target_districts:
        for city in get_cities(district):
            ''' Записи в xml выглядят таким образом:
            <item id="145181" n="Абалаково" country_id="156" country_name="Россия" 
                  district_id="303" district_name="Иркутская область" kind="T"/>
                Нужные атрибуты - это id и n (name, имя)
            '''
            cities[int(city['@id'])] = city['@n']
    write_data(cities_file_name, data=cities, is_raw=True)
    
len(cities)

2861

In [10]:
for city_id, name in cities.items():
    for uid, entry in posts.items():
        if entry['clean_name'] == name:
            posts[uid].update({'gismeteo_id': int(city_id)})

posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Усть-Камо'},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Байкит',
  'gismeteo_id': 3995},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка'

Посмотрим, сколько населённых пунктов их списка постов гидрологического контроля рек есть в базе gismeteo:

In [11]:
len(names), len(set([x['clean_name'] for x in posts.values() if 'gismeteo_id' in x.keys()]))

(25, 19)

Для 25 уникальных локаций пунктов наблюдений нашлось соответствие с 19 населёнными пунктами в Gismeteo.

### Реализация проверки наличия метео-данных

Получение исторических метео-данных происходит следующим образом:
1. Задаются все необходимые параметры - страна, область, город, месяц и год
2. Нажатие на кнопку Получить дневник выполняет перенаправление на url вида  
https://www.gismeteo.ru/diary/4036/2008/1/  
в котором:  
- **4036** - ID населённого пункта
- **2008** - год
- **1** - месяц без лидирующего нуля

Если данные о погоде за указанный период не были зафиксированы, то вместо дневника будет сообщение об отсутствии данных.

In [12]:
import os
from bs4 import BeautifulSoup
from htmlmin import minify

def get_gismeteo_table(city_id, year, month):
    file_name = os.path.join('gismeteo', str(city_id), f'{year}-{month:02d}.html')
    if is_data_exists(file_name, is_raw=True):
        weather = open_file(file_name, is_raw=True)
        soup = BeautifulSoup(weather, 'lxml')
        if soup.find(class_='empty_phrase'):
            return None
        return soup
    else:
        url = f'https://www.gismeteo.ru/diary/{city_id}/{year}/{month}/'
        r = get_url(url)
        weather = r.text
        soup = BeautifulSoup(weather, 'lxml')
        empty_phrase = soup.find(class_='empty_phrase')
        if empty_phrase:
            write_data(file_name, data=str(empty_phrase), is_raw=True)
            return None
        
        table = soup.find('table')
        # для формирования форматированных html-страниц (с отступами и переносами строк)
        # можно использовать table.prettify(), для минификации (максимально возможного
        # сокращения размера html без потери функционала) - сторонний модуль htmlmin
        write_data(file_name, data=minify(str(table)), is_raw=True)
        return table

def check_gismeteo(city_id, year):
    return get_gismeteo_table(city_id, year, 1) is not None

check_gismeteo(4036, 2008)

True

Проверяем, есть ли за первый (2008) и последний год (2017) метео-данные у выбранных городов. Если у города нет соответствия с БД gismeteo, на конечный год у него нет данных, то изменяем его gismeteo_id на -1 для дальнейшей его замены на корректный город. Если на начальный год нет данных - добавляем словарь fallback с данными о городе для замены: id города в БД gismeteo и его географические координаты, который будет использоваться для получения данных вместо основного id.

In [13]:
def is_bad_city(uid, city, print_info=False):
    if 'gismeteo_id' not in city.keys() or city['gismeteo_id'] == -1:
        if print_info:
            print(f'- {city["clean_name"]} нет в БД gismeteo')
        posts[uid].update({'gismeteo_id': -1})
        return True
    elif not check_gismeteo(city["gismeteo_id"], END_YEAR):
        if print_info:
            print(f'@ {city["clean_name"]} без метео-данных на {END_YEAR} год, нужна замена')
        posts[uid].update({'gismeteo_id': -1})
        return True
    elif not check_gismeteo(city["gismeteo_id"], START_YEAR):
        if 'fallback' in city.keys() and city['fallback']['gismeteo_id'] != -1 \
            and check_gismeteo(city['fallback']['gismeteo_id'], START_YEAR):
            if print_info:
                print(f'+ {city["clean_name"]} имеет корректную подмену')
            return False
        if print_info:
            print(f'* {city["clean_name"]} без метео-данных на {START_YEAR} год, нужна подмена')
        posts[uid].update({'fallback': {'gismeteo_id': -1}})
        return True
    elif print_info:
        print(f'+ {city["clean_name"]} имеет все необходимые метео-данные')
    return False

def get_bad_cities(print_info=False):
    result = []
    for uid, item in posts.items():
        if is_bad_city(uid, item):
            result.append(item)
    return result

bad_cities = get_bad_cities(print_info=True)
print(f'Количество постов наблюдения с проблемными населёнными пунктами: {len(bad_cities)} пунктах')
posts

Количество постов наблюдения с проблемными населёнными пунктами: 23 пунктах


{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155,
  'fallback': {'gismeteo_id': -1}},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Усть-Камо',
  'gismeteo_id': -1},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Байкит',
  'gismeteo_id': 3995},
 '09

### Замена населённых пунктов

Так как для некоторых населённых пунктов не нашлось соответствие с списком gismeteo, а также на начало 2008 года не было метео-сводок по 23 постам наблюдения, то на отсутствующие периоды времени для полноты исходных данных необходимо определить им замену. Вместо отсутствующих пунктов будем использовать их административные центры, т.к. по ним вероятнее всего будут исторические погодные данные, после чего проверим, есть ли среди заменённых городов те, которые географически расположены ближе к основному городу.

Для определения административных центров будет использован сайт Википедии, где на страницах с населёнными пунктами указан муниципальный район, к которому он принадлежит, а на странице с этим районом - административный центр.

In [14]:
def get_wiki_page(page_name):
    file_name = os.path.join('wiki', f'{page_name}.html')
    if is_data_exists(file_name, is_raw=True):
        wiki = open_file(file_name, is_raw=True)
        table = BeautifulSoup(wiki, 'lxml')
    else:
        url = f'https://ru.wikipedia.org/wiki/{page_name}'
        r = get_url(url)
        wiki = r.text
        wiki = BeautifulSoup(wiki, 'lxml')
        table = wiki.find('tbody')  # для работы нам нужна лишь таблица
        write_data(file_name, data=minify(str(table)), is_raw=True)
    return table

def get_coordinates(wiki):
    map_info = wiki.find('a', class_='mw-kartographer-maplink')
    latitude = float(map_info['data-lat'])
    longitude = float(map_info['data-lon'])
    return latitude, longitude

In [15]:
def fix_bad_cities(replace_func=None):
    bad_cities = get_bad_cities()
    print(f'Количество постов наблюдения с проблемными населёнными пунктами: {len(bad_cities)} пунктах')
    for city in bad_cities:
        try:
            def get_district(page):
                wiki = get_wiki_page(page)
                district = wiki.find('a', string="Муниципальный район")
                if not district:
                    if 'wiki_page' in city.keys() and page != city['wiki_page']:
                        #print(f"? пробуем {city['clean_name']} открыть как {city['wiki_page']}")
                        return get_district(city['wiki_page'])
                return district
            
            city_name = city['clean_name']
            district = get_district(city_name)
            if not district:
                print('- https://ru.wikipedia.org/wiki/' + city_name.replace(' ', '_'))
                continue

            district = district.parent.parent.find('td').find('a')['title']  # поиск строки с названием района
            print(f'+ {city_name} ({district})', end='')
            wiki = get_wiki_page(district)
            admin_town = wiki.find('th', string="Адм. центр")
            if not admin_town:
                print(f'\nАдминистративный центр для {city_name} ({district}) не обнаружен')
                continue
            admin_town = admin_town.parent.find('a').text
            admin_town = admin_town.replace('ё', 'е')  # в БД gismeteo нет букв Ё
            print(f' ==> {admin_town}', end='')
            admin_town_id = next((city_id for city_id, name in cities.items() if name == admin_town))
            admin_town_id = int(admin_town_id)
            print(f' ==> {int(admin_town_id)}', end='')
            for uid, post in posts.items():
                if post['clean_name'] == city_name and post['gismeteo_id'] != admin_town_id \
                    and ('fallback' not in post.keys() or post['fallback']['gismeteo_id'] != admin_town_id):
                    print(f' ==> исправлено! ', end='')
                    
                    # get gps coordinates
                    latitude, longitude = get_coordinates(wiki)
                    if replace_func:
                        admin_town = replace_func(admin_town)
                    
                    if post['gismeteo_id'] == -1:
                        posts[uid].update({'latitude': latitude, 
                                           'longitude': longitude,
                                           'gismeteo_id': admin_town_id,
                                           'wiki_page': admin_town})
                        print(f'Сделана замена')
                    else:
                        posts[uid].update({'fallback':
                                              {'latitude': latitude, 
                                               'longitude': longitude,
                                               'gismeteo_id': admin_town_id,
                                               'wiki_page': admin_town}})
                        print(f'Сделана подмена')
                    break
            
        except requests.exceptions.HTTPError:
            # Википедия даёт 404 статус на отсутствующие статьи, игнорируем
            print(f'Статья {city["clean_name"]} не найдена')
            print('- https://ru.wikipedia.org/wiki/' + city['clean_name'])
            continue

fix_bad_cities()
print(f'Число проблемных населённых пунктов после исправления: {len(get_bad_cities())}')
posts

Количество постов наблюдения с проблемными населёнными пунктами: 23 пунктах
+ Чемдальск (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Усть-Камо (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана замена
- https://ru.wikipedia.org/wiki/Кузьмовка
+ Стрелка-Чуня (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Муторай (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
- https://ru.wikipedia.org/wiki/Вельмо
- https://ru.wikipedia.org/wiki/Тея
- https://ru.wikipedia.org/wiki/Бурный
+ Суломай (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
- https://ru.wikipedia.org/wiki/Светлана
- https://ru.wikipedia.org/wiki/Вельмо
+ Верхнекарелина (Киренский район) ==> Киренск ==> 4743 ==> исправлено! Сделана замена
- https://ru.wikipedia.org/wiki/Подволошино
- https://ru.wikipedia.org/wiki/Преображенка
+ Кислокан (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
- https://ru.wikipedia.

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155,
  'fallback': {'latitude': 65.6298361,
   'longitude': 98.3047833,
   'gismeteo_id': 4015,
   'wiki_page': 'Тура'}},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Усть-Камо',
  'gismeteo_id': 4015,
  'latitude': 65.6298361,
  'longitude': 98.3047833,
  'wiki_page': 'Тура'},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'subpool_id': '114',
 

Некоторые проблемные страницы открывают список страниц с похожими названиями.

![Несколько вариантов страниц в Википедии](images/wiki_variants.png)

В таких случаях необходимо вручную задать верную страницу на Википедии.

Посёлок Светлана на карте не был обнаружен, заменим его на посёлок Вельмо.

In [16]:
# treat Светлана as Вельмо
posts['09560'].update({'gismeteo_id': posts['09396']['gismeteo_id'],
                           'fallback': posts['09396']['fallback']})

def fix_name(name):
    # Бакланиха  - ближайший к реке Большой порог населённый пункт
    result = name.replace('Большой Порог', 'Бакланиха_(Красноярский_край)')
    result = result.replace('Бурный', 'Бурный_(Эвенкийский_район)')
    result = result.replace('Вельмо', 'Вельмо_(посёлок)')
    result = result.replace('Кузьмовка', 'Кузьмовка_(Красноярский_край)')
    result = result.replace('Подволошино', 'Подволошино_(Иркутская_область)')
    result = result.replace('Преображенка', 'Преображенка_(Катангский район)')
    result = result.replace('Светлана', 'Вельмо_(посёлок)')
    result = result.replace('Тура', 'Тура_(Красноярский_край)')
    result = result.replace('Тембенчи', 'Тура_(Красноярский_край)')
    result = result.replace('Тея', 'Тея_(Северо-Енисейский_район)')
    result = result.replace('Токма', 'Токма_(село)')
    result = result.replace('Учами', 'Учами_(Полигусовский_сельсовет)')
    result = result.replace('Ика', 'Ика_(село)')
    result = result.replace('Непа', 'Непа_(село)')
    return result

for uid, post in posts.items():
    new_name = fix_name(post['clean_name'])
    if post['clean_name'] != new_name:
        print(f"{post['clean_name']} теперь {new_name}")

    posts[uid].update({'wiki_page': new_name})
    
    if 'fallback' in post.keys() and 'wiki_page' in post['fallback'].keys():
        new_name = fix_name(post['fallback']['wiki_page'])
        if post['fallback']['wiki_page'] != new_name:
            print(f"wiki: {post['fallback']['wiki_page']} теперь {new_name}")
            
        posts[uid]['fallback'].update({'wiki_page': new_name})
        

wiki: Тура теперь Тура_(Красноярский_край)
Кузьмовка теперь Кузьмовка_(Красноярский_край)
wiki: Тура теперь Тура_(Красноярский_край)
wiki: Тура теперь Тура_(Красноярский_край)
Вельмо теперь Вельмо_(посёлок)
Тея теперь Тея_(Северо-Енисейский_район)
Бурный теперь Бурный_(Эвенкийский_район)
wiki: Тура теперь Тура_(Красноярский_край)
Светлана теперь Вельмо_(посёлок)
Вельмо теперь Вельмо_(посёлок)
Подволошино теперь Подволошино_(Иркутская_область)
Преображенка теперь Преображенка_(Катангский район)
wiki: Тура теперь Тура_(Красноярский_край)
Тура теперь Тура_(Красноярский_край)
Учами теперь Учами_(Полигусовский_сельсовет)
Большой Порог теперь Бакланиха_(Красноярский_край)
Токма теперь Токма_(село)
Ика теперь Ика_(село)
Тембенчи теперь Тура_(Красноярский_край)
Большой Порог теперь Бакланиха_(Красноярский_край)
Непа теперь Непа_(село)


Повторно найдём замену для проблемных городов:

In [17]:
fix_bad_cities(replace_func=fix_name)
print(f'Число проблемных населённых пунктов после исправления: {len(get_bad_cities())}')

Количество постов наблюдения с проблемными населёнными пунктами: 14 пунктах
+ Кузьмовка (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Вельмо (Северо-Енисейский район) ==> Северо-Енисейский ==> 4006 ==> исправлено! Сделана подмена
+ Тея (Северо-Енисейский район) ==> Северо-Енисейский ==> 4006 ==> исправлено! Сделана подмена
+ Бурный (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Светлана (Северо-Енисейский район) ==> Северо-Енисейский ==> 4006 ==> исправлено! Сделана подмена
+ Вельмо (Северо-Енисейский район) ==> Северо-Енисейский ==> 4006 ==> исправлено! Сделана подмена
+ Подволошино (Катангский район Иркутской области) ==> Ербогачен ==> 4031 ==> исправлено! Сделана подмена
+ Преображенка (Катангский район) ==> Ербогачен ==> 4031 ==> исправлено! Сделана подмена
+ Учами (Эвенкийский район) ==> Тура ==> 4015 ==> исправлено! Сделана подмена
+ Большой Порог (Туруханский район) ==> Туруханск ==> 3975 ==> исправлено! Сделана замена
+ Ика (Ка

Найдём координаты всех городов:

In [18]:
def fill_gps_info():
    for uid, city in posts.items():
        if is_bad_city(uid, city):
            continue
        wiki = get_wiki_page(city['wiki_page'])
        latitude, longitude = get_coordinates(wiki)
        posts[uid].update({'latitude': latitude, 'longitude': longitude})
        
fill_gps_info()
posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155,
  'fallback': {'latitude': 65.6298361,
   'longitude': 98.3047833,
   'gismeteo_id': 4015,
   'wiki_page': 'Тура_(Красноярский_край)'},
  'wiki_page': 'Чемдальск',
  'latitude': 59.62583,
  'longitude': 103.32778},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036,
  'wiki_page': 'Ванавара',
  'latitude': 60.34528,
  'longitude': 102.28417},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Усть-Камо',
  'gi

Проверим, если ли более близкие города к основному городу, чем ранее найденные административные центры:

In [19]:
# считаем кратчайшее расстояние по большому кругу
from geopy.distance import great_circle, lonlat

for uid, post in posts.items():
    if 'fallback' in post.keys():
        orig_lonlat = lonlat(post['longitude'], post['latitude'])
        fallback_lonlat = lonlat(post['fallback']['longitude'], post['fallback']['latitude'])
        orig_distance = great_circle(orig_lonlat, fallback_lonlat)
        
        min_distance = orig_distance
        distance = orig_distance
        nearest_city = None
        need_skip = False
        for comp_post in posts.values():
            if 'fallback' in comp_post.keys() and \
                    comp_post['fallback']['gismeteo_id'] != post['fallback']['gismeteo_id'] and \
                    comp_post['fallback']['gismeteo_id'] != post['gismeteo_id']:
                test_lonlat = lonlat(comp_post['fallback']['longitude'], comp_post['fallback']['latitude'])
                distance = great_circle(orig_lonlat, test_lonlat)
                if distance < min_distance and distance > 0.0:
                    min_distance = distance
                    nearest_city = comp_post['fallback']
            elif comp_post['gismeteo_id'] != post['fallback']['gismeteo_id'] and \
                    comp_post['gismeteo_id'] != post['gismeteo_id']:
                test_lonlat = lonlat(comp_post['longitude'], comp_post['latitude'])
                distance = great_circle(orig_lonlat, test_lonlat)
                if distance < min_distance and distance > 0.0:
                    min_distance = distance
                    nearest_city = comp_post
        if nearest_city is not None:
            print(f"{post['wiki_page']}: {nearest_city['wiki_page']} на {round(orig_distance.km - min_distance.km)} км " +
                  f"ближе, чем свой запасной {post['fallback']['wiki_page']}")
            #print(post)
            #print(nearest_city)
            posts[uid]['fallback'].update({'latitude': nearest_city['latitude'],
               'longitude': nearest_city['longitude'],
               'gismeteo_id': nearest_city['gismeteo_id'],
               'wiki_page': nearest_city['wiki_page']})

Чемдальск: Ванавара на 616 км ближе, чем свой запасной Тура_(Красноярский_край)
Кузьмовка_(Красноярский_край): Бурный_(Эвенкийский_район) на 414 км ближе, чем свой запасной Тура_(Красноярский_край)
Стрелка-Чуня: Муторай на 355 км ближе, чем свой запасной Тура_(Красноярский_край)
Муторай: Стрелка-Чуня на 359 км ближе, чем свой запасной Тура_(Красноярский_край)
Бурный_(Эвенкийский_район): Кузьмовка_(Красноярский_край) на 445 км ближе, чем свой запасной Тура_(Красноярский_край)
Суломай: Бурный_(Эвенкийский_район) на 488 км ближе, чем свой запасной Тура_(Красноярский_край)
Подволошино_(Иркутская_область): Верхнекарелина на 250 км ближе, чем свой запасной Ербогачен
Преображенка_(Катангский район): Непа_(село) на 21 км ближе, чем свой запасной Ербогачен
Кислокан: Стрелка-Чуня на 138 км ближе, чем свой запасной Тура_(Красноярский_край)
Учами_(Полигусовский_сельсовет): Кузьмовка_(Красноярский_край) на 308 км ближе, чем свой запасной Тура_(Красноярский_край)
Ика_(село): Непа_(село) на 101 км бл

Теперь для всех постов наблюдения можно получить полные метео-данные за указанный период.

Сохраним словарь с данными о пунктах наблюдений.

In [20]:
write_data(DATA_POSTS_FULL_RAW, data=posts, is_raw=True)
posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Чемдальск',
  'gismeteo_id': 158155,
  'fallback': {'latitude': 60.34528,
   'longitude': 102.28417,
   'gismeteo_id': 4036,
   'wiki_page': 'Ванавара'},
  'wiki_page': 'Чемдальск',
  'latitude': 59.62583,
  'longitude': 103.32778},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Ванавара',
  'gismeteo_id': 4036,
  'wiki_page': 'Ванавара',
  'latitude': 60.34528,
  'longitude': 102.28417},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'subpool_id': '114',
  'subpool_name': 'Подкаменная Тунгуска',
  'cadid': '116103387',
  'water_site': 'Подкаменная Тунгуска',
  'clean_name': 'Усть-Камо',
  'gismeteo_id': 4015,
 

## Получение и сохранение метео-данных

Напишем функцию, которая запрашивает и обрабатывает html-страницу с метео-данными:

In [21]:
'''            url: https://www.gismeteo.ru/diary/4036/2008/1/
columns: [0]   [1]   [2]   [3]     [4]    [5]   [6]   [7]   [8]    [9]     [10]
row [0]      |             день               |            вечер               |
row [1] число|темп.|давл.|облач.|явления|ветер|темп.|давл.|облач.|явления|ветер|
       ------------------------------------------------------------------------|
row [2]   1  | -15 | 732 | img  |  img  |З 2мс| -14 | 733 | img  |  img  |З 2мс|
      -------------------------------------------------------------------------|
На некоторые дни нет метео-данных, вместо них - прочерки
Пример - 19 числа на https://www.gismeteo.ru/diary/4015/2010/12/
''' 
def process_row(row):
    cells = row.find_all('td')
    
    # day, temperature
    def get_cell_int(cell_id):
        if cell_id > 10:
            return None
        val = cells[cell_id].text
        if val == '−' or len(val) == 0:
            return get_cell_int(cell_id + 5)  # если за день нет информации, то берём её за вечер
        #print(val)
        return int(val)
        
    day = get_cell_int(0)
    temperature = get_cell_int(1)
    
    # cloud, weather
    def get_cell_img(cell_id):
        if cell_id > 10:
            return None
        imgs = cells[cell_id].find_all('img')
        if len(imgs) > 0:
            if len(imgs) > 2:  # обычно в ячейке находятся 2 картинки - цветная и ч/б
                raise Exception(f"больше, чем 2 картинки")
            img = imgs[0]['src']  # //st6.gismeteo.ru/static/diary/img/dull.png
            if img.endswith('still.gif'):
                return get_cell_img(cell_id + 5)
            return re.search(r"diary/img/(\w+).png", img).group(1)
        return 'clear'
    
    cloud = get_cell_img(3)
    if cloud == 'clear':
        cloud = None
    weather = get_cell_img(4)

    # в реальных долгосрочных прогнозах погоды на весь день даётся 1 прогноз, который содержит
    # минимальную и максимальную температуру, облачность и наличие осадков.
    
    return [day, temperature, cloud, weather]

table = get_gismeteo_table(4036, 2008, 1)
data_rows = table.find_all('tr')
data_rows = data_rows[2:]  # убрать шапку таблицы
result = []
for row in data_rows:
    dict_item = process_row(row)
    result.append(dict_item)
print(dict_item)

[31, -17, 'sunc', 'clear']


Теперь сделаем функцию, которая будет сохранять метео-данные в формате csv:

In [22]:
from datetime import datetime
import csv
import os
from calendar import monthrange

# id_поста,дата,широта,долгота,температура,
# облачность,осадки,использован_альтернативный_город
header = ['uid', 'date', 'latitude', 'longitude', 'temperature',
          'cloud', 'weather', 'is_fallback_data']

# Для данных параметров все необходимые страницы gismeteo будут загружаться минимум 4 часа

# Для ускорения процесса парсинга сайта можно использовать прокси и многопоточность через
# встроенную библиотеку multiprocessing, но для этичности проекта и избежания излишней 
# нагрузки на сайт запросы будут производиться в однопоточном режиме

def make_dataset(file_name, start_year, end_year):
    result = []
    if is_data_exists(file_name, is_raw=True):
        print(f"Датасет {file_name} уже сформирован.")
    else:
        for uid, post in posts.items():
            print(f"{post['name']} - год ", end='')
            for year in range(start_year, end_year + 1):        
                print(f"{year}… ", end='')
                for month in range(1, 13):
                    def get_data_rows(city_id):
                        table = get_gismeteo_table(city_id, year, month)
                        if not table:
                            return None

                        data_rows = table.find_all('tr')
                        return data_rows[2:]  # убрать шапку таблицы

                    # В приоритете используются метео-данными из основного источника, если их
                    # Примеры неполных метеоданных:
                    # с 18 + пропуски https://www.gismeteo.ru/diary/158155/2015/3/
                    # до 19 числа https://www.gismeteo.ru/diary/4015/2015/9/
                    # с 29 по 31 https://www.gismeteo.ru/diary/4015/2015/10/
                    main_data = get_data_rows(post['gismeteo_id'])
                    main_gps = [post['latitude'], post['longitude']]
                    fallback_dict = None
                    fallback_gps = None

                    if not main_data or (len(main_data) < monthrange(year, month)[1] \
                                         and 'fallback' in post.keys()):
                        fallback_gps = [post['fallback']['latitude'], post['fallback']['longitude']]
                        fallback_data = get_data_rows(post['fallback']['gismeteo_id'])
                        if not fallback_data:
                            continue  # в течении месяца нет информации ни у оригинального источника, ни у дополнительного
                        if main_data:
                            fallback_data = [process_row(row) for row in fallback_data]
                            fallback_dict = {day: (temperature,  cloud, weather, 1)  # 1 is is_fallback_data
                                            for day, temperature, cloud, weather in fallback_data}
                        else:
                            main_data = fallback_data
                            main_gps = fallback_gps
                    
                    is_fallback = int(main_gps == fallback_gps or post['clean_name'] == 'Светлана')  # check if it is Светлана
                    day = 1 
                    for row in main_data:
                        try:
                            data = process_row(row)
                        except Exception:
                            err_str = f"check https://www.gismeteo.ru/diary/{post['gismeteo_id']}/{year}/{month}/"
                            if 'fallback' in post.keys():
                                err_str += f"\n+ https://www.gismeteo.ru/diary/{post['fallback']['gismeteo_id']}/{year}/{month}/"
                            raise Exception(err_str)

                        row_day = data[0]
                        while day != row_day:  # есть пропуски в днях в основном источнике метео-данных
                            if fallback_dict:  # заполняем пропуски из второстепенного источника
                                #print(f'\nuse fallback to fill {day} day')
                                if day in fallback_dict:
                                    date_string = f'{year}-{month:02d}-{day:02d}'
                                    result.append([uid, date_string] + fallback_gps + list(fallback_dict[day]))
                                    #print(fallback_dict[day])
                                    day += 1
                                    continue

                            # в обоих источниках есть пропуски данных - игнорируем
                            day += 1

                        date_string = f'{year}-{month:02d}-{row_day:02d}'
                        result.append([uid, date_string] + main_gps + data[1:] + [is_fallback])
                        day += 1
            print()  # перенос строки
        write_csv(file_name, header, result, is_raw=True)

make_dataset(DATA_WEATHER, START_YEAR, END_YEAR)
print('completed')

Датасет weather.csv уже сформирован.
completed
